In [ ]:
#I learned a lot and implemented from the bottom notebooks
#https://www.kaggle.com/startupsci/titanic-data-science-solutions
#https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
#Do not forget to visit them 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


%matplotlib inline
import matplotlib.pyplot as plt  # Matlab stilinde grafik çizmek için
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #Sklearn ve seaborn'un verdiği cansıkıcı uyarıları engellemek için


from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Float çıktıları 3 basamakla sınırlandırmak için

In [ ]:
#Dosyaları yükle
#Train tüm verilerimiz. Test ise belli bir kısmı
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
#ilk 5 elemanı göster
train.head(5)

In [ ]:
#İlk 5 elemanı göster
test.head(5)

In [ ]:
#boyutlarına bak
train.shape

In [ ]:
#boyutlarına bak
test.shape

In [ ]:
#'Id' sütununu kaydet
train_ID = train['PassengerId']
test_ID = test['PassengerId']

In [ ]:
#'Id' sütunu gereksiz bu yüzden sil
train.drop("PassengerId", axis = 1, inplace = True)
test.drop("PassengerId", axis = 1, inplace = True)

In [ ]:
#silme başarılı
train.shape

In [ ]:
#silme başarılı
test.shape

In [ ]:
#Data Processing
#Aykırı değerler

In [ ]:
#Özellik mühendisliği

# Train ve test verilerini aynı dataframe içerisine birleştiriyoruz
ntrain = train.shape[0]
ntest = test.shape[0]

y_train = train.Survived.values
#birleştirme işlemi
all_data = pd.concat((train, test)).reset_index(drop=True)
#'Survived' sütununu siliyoruz
all_data.drop(['Survived'], axis=1, inplace=True)

#Tüm datanın büyüklüğü
print("all_data büyüklüğü : {}".format(all_data.shape))

In [ ]:
#Eksik veriler
#Tüm veriye oranla eksiklik yüzdesi veriyoruz
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
#Eksik veriler adında bir tablo oluşturuyoruz
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
#İlk 20 satıra bakıyoruz
missing_data.head(20)

In [ ]:
#Eksik veriyi tabloda gösteriyoruz
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Özellikler', fontsize=15)
plt.ylabel('Eksik değer oranı', fontsize=15)
plt.title('Özellik başına eksik veri oranı', fontsize=15)

In [ ]:
#İlginçtir ki burda sınıfla güçlü bir ilişki bekliyordum ancak öyle çıkmadı. Bunun sebebinin sınıflardaki kişi
#sayısı farkı ile ilgili olabileceğini düşünüyorum. Bilete ödenen ücretle hayatta kalmanın güçlü ilişkisi bunu doğrular
#nitelikte.
#Veri korelasyonu
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
#Ölen ve yaşayanların yaş ile ilişkilerine bakıyorum
g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
#Sınıf ile hayatta kalma durumları ilişkisi
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
all_data['Name'] = all_data['Name'].map(lambda x: re.sub('\W', '', x))
all_data['Name'] = all_data['Name'].map(lambda x: re.sub('\W', '', x))
all_data['Name'] = all_data['Name'].map(lambda x: re.sub('\W', '', x))

In [ ]:
#Korelasyondan 'Parch' ile 'SibSp' sayısının aynı şeyi gösterdiğini anlıyoruz. Mantık olarak düşününce de bunu kanıtlar nitelikte
#Bu nedenle bir tanesini sileceğim
all_data = all_data.drop(['Parch'], axis=1)

In [ ]:
#Fare(Bilet parası): %0.076'lık çok az eksik veri söz konusu, bilet parasının da sınıfa bağımlı bir değişken 
#olduğunu bildiğimiz için o sınıftaki bilet ücretlerinin medyanını atıyoruz.
all_data["Fare"] = all_data.groupby("Pclass")["Fare"].transform(lambda x: x.fillna(x.median()))

In [ ]:
#Cabin: Çok büyük kısmının(%77.4) eksik veri olması ve kabin numarasının bir fayda sağlamayacağı kanısı ile silmeye karar veriyorum.
all_data = all_data.drop(['Cabin'], axis=1)

In [ ]:
#Age: Büyük kısmı eksik(%20).Burası en önemli karar noktalarından biri. Emin olmamakla beraber sınıfın yaş medyanını atamaya karar veriyorum.
all_data["Age"] = all_data.groupby("Pclass")["Age"].transform(lambda x: x.fillna(x.median()))

In [ ]:
#Embarked: Gemiye nereye bindikleri. Çok az değer eksiği var. Kendi sınıfında en sık görüleni atayacağım(Kategorik veri).
all_data['Embarked'] = all_data['Embarked'].fillna(all_data['Embarked'].mode()[0])

In [ ]:
#Geriye kalan eksik değer var mı diye kontrol edilir
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

In [ ]:
#Embarked ve Sex değişkenini kategorikten sayısala dönüştüreceğiz.
number = LabelEncoder()
all_data["Sex"]= number.fit_transform(all_data["Sex"].astype('str'))
all_data["Embarked"]= number.fit_transform(all_data["Embarked"].astype('str'))


In [ ]:
#Çarpık değerler
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Tüm sayısal değerlerdeki çarpıklığa bakıyoruz
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nNumerical özelliklerdeki çarpıklık: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

In [ ]:
#The more skewed the data, the less accurate model will be.
from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

#Sahte kategorik verileri alıyoruz
all_data = pd.get_dummies(all_data)
print(all_data.shape)
#Yeni train ve test kümelerini alıyoruz
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
#Modelleme
#Kütüphaneleri ekliyoruz
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC # lineer modellerimiz
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error #Modellerin başarısını ölçmek için
import xgboost as xgb # Son zamanlarda yüksek başarı gösteren bir model
import lightgbm as lgb 

In [ ]:
#Define a cross validation strategy
#cross_val_score fonksiyonunu Sklearn kütüphanesinden kullanıyoruz ama kendi içerisinde karıştırmayı
# bulundurmuyor bu nedenle biz ayrıca manuel karıştırmayı sağlıyoruz
#Doğrulama fonksiyonu
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
#Modellerin temelleri

In [ ]:
#Lasso regresyon aykırı değerlere karşı hassas olduğu için o değerlere karşı güçlendireceğiz.
#Bunu sklearn'ün Robustscaler() fonksiyonu ile yapacağız.
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
#ElasticNet regresyonu da aynı şekilde güçlendiriyoruz.
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
#Kernel Ridge Regresyonu:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
#Gradient Boosting Regresyon :
#Aykırı değerlere karşı sağlam kılan huber kaybı ile birlikte.
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
#XGBoost: Ölçeklenebilir, Taşınabilir ve Dağıtılmış Degrade Artırıcı Kütüphane" sağlamayı amaçlamaktadır
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
#LightGBM: LightGBM, Light Gradient Boosting Machine'in kısaltmasıdır, ilk olarak Microsoft tarafından geliştirilen
#makine öğrenimi için ücretsiz ve açık kaynaklı dağıtılmış bir gradyan artırma çerçevesidir. 
#Karar ağacı algoritmalarına dayanır ve sıralama, sınıflandırma ve diğer makine öğrenimi görevleri için kullanılır.
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                               learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                             bagging_freq = 5, feature_fraction = 0.2319,
                            feature_fraction_seed=9, bagging_seed=9,
                            min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
#Model skorları
#Temel modellerimizin  cross-validation rmsle(çapraz onaylama kök ortalamalar logaritmik karesi)
# hata ölçme metoduna göre başarılarını ölçeceğiz
#İşlem birkaç dakika alabilir
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

print("Done")

In [ ]:
#Xgboost score: 0.3648 (0.0240)
#LGBM score: 0.3658 (0.0234)
#Lasso score: 0.3819 (0.0233)
#ElasticNet score: 0.3807 (0.0228)


In [ ]:
#En başarılı ilk 4 modeli alıp yığıcam
#Modelleri istifleme
#Veri biliminde aynı modeli defalarca kullanmak bir fayda sağlamazken
#farklı modelleri bir araya getirmek başarı sağlayabilmektedir*
#Basit yığıtlama yaklaşımı: Temel modellerin ortalamasını almak
#Scikit-learn'ü kendi modelimizle genişletmek(inheritence(miraslama) ve encapsulation(kapsülleme) yapmak)
#için yeni bir sınıf oluşturuyoruz

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    #constructor
    def __init__(self, models):
        self.models = models
        
    # Veriyi sığdırmak için orijinal modelin klonlarını oluşturuyoruz
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Klonlanmış modelleri eğitiyoruz
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Klonlanmış modeller için tahmin yapıyoruz ve ortalamasını alıyoruz
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
#Sadece 4 modelin ortalamasını alacağız isteseydik daha da fazla alabilirdik.ENet, GBoost, KRR and lasso alacağız.
averaged_models = AveragingModels(models = (model_xgb,model_lgb, lasso,ENet))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # Orijinal modelin klonları üzerine veriyi ekliyoruz
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        #Klon modelleri eğit sonra klonlanmış meta modelleri eğitmek için gereken
        #out-of-fold(katlama dışı kalan) tahminleri yarat 
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Klonlanmış modelleri out-of-fold tahminleri kullanarak yeni bir özellik olacak şekilde eğit
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Test verilerindeki tüm temel modellerin tahminlerini yapın ve ortalama tahminleri meta-model tarafından yapılan 
    #tahminler için meta özellikler olarak kullanın
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
#Her iki model yığıtlama modelini karşılaştırabilmek için aynı modelleri girdi olarak kullanacağız
#İşlem birkaç dakika sürebilir
stacked_averaged_models = StackingAveragedModels(base_models = (model_xgb, GBoost, model_lgb),
                                                 meta_model = ENet)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
#stacked regressor:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
#Sonuçların olduğu bir tablo yapıyoruz ve bitiriyoruz
sub = pd.DataFrame()
sub['PassengerId'] = test_ID
sub['Survived'] = ensemble
sub.to_csv('submission.csv',index=False)